In [1]:
!nvidia-smi

Mon Nov  2 20:33:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -qq wandb
!git clone https://github.com/lattice-ai/GPT
%cd GPT

/content/GPT


In [3]:
from gpt.experiments.language_model import IMDBReviewLanguageExperiment

experiment = IMDBReviewLanguageExperiment()

In [4]:
from gpt.experiments.utils import init_wandb

init_wandb(
    project_name='gpt', experiment_name='imdb_language_model',
    wandb_api_key='41442f74323655b7ec081f4af030fa821f294c72'
)

wandb: Currently logged in as: lattice-ai (use `wandb login --relogin` to force relogin)


In [5]:
experiment.build_dataset(
    'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz')

experiment.compile()

Dataset Size: 50000 files
Dataset: <PrefetchDataset shapes: ((None, 100), (None, 100)), types: (tf.int64, tf.int64)>
Model: "GPT_depth_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GPT_Input (InputLayer)       [(None, 100)]             0         
_________________________________________________________________
combined_encoding (CombinedE (None, 100, 256)          5145600   
_________________________________________________________________
transformer_block (Transform (None, 100, 256)          395776    
_________________________________________________________________
GPT_Output (Dense)           (None, 100, 20000)        5140000   
Total params: 10,681,376
Trainable params: 10,681,376
Non-trainable params: 0
_________________________________________________________________


In [6]:
start_text = 'the actor was'
start_tokens = experiment.tokenize(start_text=start_text)
experiment.train(
    epochs=30, start_tokens=start_tokens,
    max_length=100, max_tokens=40, top_k=10,
    infer_every=1, log_on_wandb=True
)

Epoch 1/30
   1563/Unknown - 144s 92ms/step - loss: 5.0512 - GPT_Output_loss: 5.0512
Sample Generate Text: the actor was very believable . . . . . the film is very good , the plot is not even the movie itself . it 's very hard to get a bit of it . but the plot has a very bad taste very believable . . . . . the film is very good , the plot is not even the movie itself . it 's very hard to get a bit of it . but the plot has a very bad taste

Epoch 00001: saving model to wandb/run-20201102_203410-3lk64c01/files/gpt_language_model_checkpoint1
1563/1563 [==============================] - 146s 93ms/step - loss: 5.0512 - GPT_Output_loss: 5.0512
Epoch 2/30
1563/1563 [==============================] - ETA: 0s - loss: 4.4895 - GPT_Output_loss: 4.4895
Sample Generate Text: the actor was a [UNK] , this is a very good film . the acting was terrible . . it was bad and bad . it was bad . . . . . .the film is good enough , the acting , the acting a [UNK] , this is a very good film . the acting was ter

KeyboardInterrupt: ignored

In [7]:
start_text = 'the movie is'
start_tokens = experiment.tokenize(start_text=start_text)
output = experiment.infer(
    start_tokens, max_length=100, max_tokens=40, top_k=10
)
print('Generated Text:', output)

Generated Text: the movie is excellent and the plot is interesting , and the acting is quite good and the plot is quite predictable . the plot is simple . it is not the acting . the only good character is that the script is so excellent and the plot is interesting , and the acting is quite good and the plot is quite predictable . the plot is simple . it is not the acting . the only good character is that the script is so
